In [1]:
import numpy as np

def black_scholes_crank_nicolson(S0, K, T, r, sigma, dt, ds, option_type='call'):
    """
    Prices an option using the Crank-Nicolson method for the Black-Scholes PDE.

    Args:
        S0: Initial stock price
        K: Strike price
        T: Time to maturity
        r: Risk-free interest rate
        sigma: Volatility
        dt: Time step
        ds: Stock price step
        option_type: 'call' or 'put'

    Returns:
        Option price
    """

    # Grid setup
    S_max = S0 * np.exp(3 * sigma * np.sqrt(T))
    S_min = S0 * np.exp(-3 * sigma * np.sqrt(T))
    S_grid = np.arange(S_min, S_max + ds, ds)
    t_grid = np.arange(0, T + dt, dt)

    # Boundary conditions
    if option_type == 'call':
        V_boundary_top = S_grid - K
        V_boundary_bottom = np.zeros_like(S_grid)
    elif option_type == 'put':
        V_boundary_top = np.zeros_like(S_grid)
        V_boundary_bottom = K - S_grid

    # Initialize the option price matrix
    V = np.zeros((len(t_grid), len(S_grid)))

    # Set the final time step (terminal condition)
    V[-1, :] = np.maximum(S_grid - K, 0) if option_type == 'call' else np.maximum(K - S_grid, 0)

    # Crank-Nicolson iteration
    for j in range(len(t_grid) - 2, -1, -1):
        A = np.zeros((len(S_grid), len(S_grid)))
        b = np.zeros(len(S_grid))

        # Construct the tridiagonal matrix A and the right-hand side vector b
        for i in range(1, len(S_grid) - 1):
            A[i, i - 1] = -0.25 * sigma**2 * S_grid[i]**2 / ds**2 - 0.5 * r * S_grid[i] / (2 * ds)
            A[i, i] = 1 + 0.5 * dt * (sigma**2 * S_grid[i]**2 / ds**2 + r)
            A[i, i + 1] = 0.25 * sigma**2 * S_grid[i]**2 / ds**2 + 0.5 * r * S_grid[i] / (2 * ds)

            b[i] = V[j + 1, i] + 0.25 * dt * (sigma**2 * S_grid[i]**2 * (V[j + 1, i + 1] - 2 * V[j + 1, i] + V[j + 1, i - 1]) / ds**2 + r * S_grid[i] * (V[j + 1, i + 1] - V[j + 1, i - 1]) / (2 * ds))

        # Set boundary conditions for A and b
        A[0, 0] = 1
        A[0, 1] = 0
        b[0] = V_boundary_top[0]

        A[-1, -1] = 1
        A[-1, -2] = 0
        b[-1] = V_boundary_bottom[-1]

        # Solve the system of equations
        V[j, :] = np.linalg.solve(A, b)

    # Find the option price at the initial stock price S0
    option_price = V[0, np.where(np.abs(S_grid - S0) == np.min(np.abs(S_grid - S0)))[0][0]]

    return option_price

In [2]:
import numpy as np

# Parameters
S0 = 100  # Initial stock price
K = 100  # Strike price
T = 1  # Time to maturity (in years)
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Volatility
dt = 0.01  # Time step
ds = 1  # Stock price step

# Calculate the option price using the Crank-Nicolson method
call_price = black_scholes_crank_nicolson(S0, K, T, r, sigma, dt, ds, option_type='call')
put_price = black_scholes_crank_nicolson(S0, K, T, r, sigma, dt, ds, option_type='put')

print("Call price:", call_price)
print("Put price:", put_price)

Call price: -30.249376036097217
Put price: 8.27493668128338
